In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver

In [2]:
url = 'https://www.latam.com/es_ar/apps/personas/booking?fecha1_dia=25&fecha1_anomes=2019-10&fecha2_dia=12&fecha2_anomes=2019-11&from_city2=MAD&to_city2=BUE&auAvailability=1&ida_vuelta=ida_vuelta&vuelos_origen=Buenos%20Aires&from_city1=BUE&vuelos_destino=Madrid&to_city1=MAD&flex=1&vuelos_fecha_salida_ddmmaaaa=25/10/2019&vuelos_fecha_regreso_ddmmaaaa=12/11/2019&cabina=Y&nadults=1&nchildren=0&ninfants=0&cod_promo='

In [3]:
agent = {"User-Agent":"Mozilla/5.0"}
r = requests.get(url, headers=agent)

In [4]:
r.status_code

200

In [5]:
s = BeautifulSoup(r.text, 'lxml')

In [6]:
from tqdm import tqdm

In [32]:
url='https://www.latam.com/es_ar/apps/personas/booking?fecha1_dia=17&fecha1_anomes=2021-04&fecha2_dia=24&fecha2_anomes=2021-09&otras_ciudades=&num_segmentos_interfaz=2&tipo_paso1=caja&rand_check=486.3230058787316&from_city2=MAD&to_city2=BUE&auAvailability=1&ida_vuelta=ida_vuelta&vuelos_origen=Buenos%20Aires,%20Argentina%20(BUE)&from_city1=BUE&vuelos_destino=Madrid,%20Espa%F1a%20(MAD)&to_city1=MAD&flex=1&vuelos_fecha_salida=27/DIC/2020&vuelos_fecha_salida_ddmmaaaa=27/12/2020&vuelos_fecha_regreso=21/ENE/2021&vuelos_fecha_regreso_ddmmaaaa=21/01/2021&cabina=Y&nadults=1&nchildren=0&ninfants=0&application=#/'
driver = webdriver.Safari()
driver.get(url)

Extraer la información de la página

In [8]:
vuelos = driver.find_elements_by_xpath('//li[@class="flight"]')

In [9]:
vuelos

[<selenium.webdriver.remote.webelement.WebElement (session="654FEAA4-D8F4-46AB-A968-1B506B600D10", element="node-D567A037-276F-423D-81D0-AEAAE6038574")>,
 <selenium.webdriver.remote.webelement.WebElement (session="654FEAA4-D8F4-46AB-A968-1B506B600D10", element="node-FEF7170E-CDEA-4EF0-9916-D9E15934B78F")>,
 <selenium.webdriver.remote.webelement.WebElement (session="654FEAA4-D8F4-46AB-A968-1B506B600D10", element="node-B1B008B4-A6EC-4DEF-8888-5D2D817645F1")>,
 <selenium.webdriver.remote.webelement.WebElement (session="654FEAA4-D8F4-46AB-A968-1B506B600D10", element="node-E996BF86-FF75-476A-A6B4-8DA00D039131")>,
 <selenium.webdriver.remote.webelement.WebElement (session="654FEAA4-D8F4-46AB-A968-1B506B600D10", element="node-82DC170B-278C-47DF-8324-307B134B5F18")>,
 <selenium.webdriver.remote.webelement.WebElement (session="654FEAA4-D8F4-46AB-A968-1B506B600D10", element="node-A184B999-3608-45D2-B7C1-1D8EF1B1C840")>,
 <selenium.webdriver.remote.webelement.WebElement (session="654FEAA4-D8F4-46

In [10]:
vuelo = vuelos[0]
vuelo

<selenium.webdriver.remote.webelement.WebElement (session="654FEAA4-D8F4-46AB-A968-1B506B600D10", element="node-D567A037-276F-423D-81D0-AEAAE6038574")>

#### Hora de llegada, hora de salida y duración 

In [11]:
# Hora de salida
salida = vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')
salida

'18:00'

In [12]:
# Hora de llegada
llegada = vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')
llegada

'14:10'

In [13]:
# Duración
duracion = vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')
formatear = duracion.replace('PT','')
formatear

'15H10M'

### Escalas 

La cantidad de escalas aparece en un `<div class='flight-summary-stops-description'>`

In [16]:
boton_escalas = vuelo.find_element_by_xpath('.//div[@class ="flight-summary-stops-description"]/button')

In [17]:
boton_escalas

<selenium.webdriver.remote.webelement.WebElement (session="654FEAA4-D8F4-46AB-A968-1B506B600D10", element="node-C9028D44-8643-4B61-9BBE-73CCC7E5297C")>

Damos click al botón

In [18]:
boton_escalas.click()

In [19]:
segmentos = vuelo.find_elements_by_xpath('//div[@class="sc-hZSUBg gfeULV"]/div[@class="sc-cLQEGU hyoued"]')
segmentos

[<selenium.webdriver.remote.webelement.WebElement (session="654FEAA4-D8F4-46AB-A968-1B506B600D10", element="node-1CD31689-C4EB-47C4-B7B1-9E0D03D4C41A")>,
 <selenium.webdriver.remote.webelement.WebElement (session="654FEAA4-D8F4-46AB-A968-1B506B600D10", element="node-B22152D8-968D-4B73-A9CB-5B9F66F050CD")>]

In [20]:
escalas = len(segmentos) - 1 #0 escalas si es un vuelo directo

Analizamos solo un segmento de la lista

In [21]:
segmento = segmentos[0]

In [ ]:
segmento.get_attribute('class')

In [22]:
segmento.text

'18:00EZESalida a las 18 Horas 00 Minutos, de Ezeiza Intl.Ezeiza Intl.2h h 50min min de duración20:50GRULlegada a las 20 Horas 50 Minutos, a Guarulhos Intl.Guarulhos Intl.Vuelo LA8013Airbus 320-200Operado por LATAM Airlines Brasil'

De cada segmento necesitamos:
- origen
- hora de salida
- destino
- hora de llegada
- duración del vuelo
- numero de vuelo
- modelo de avion
- duración de la escala

In [23]:
# Origen
segmento.find_element_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/abbr[@class="sc-hrWEMg hlCkST"]').text

'EZE'

In [24]:
# Duracion
segmento.find_element_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/time').text

'18:00'

Cierro el pop-up

In [ ]:
driver.find_element_by_xpath('//div[@class="modal-content sc-iwsKbI eHVGAN"]//button[@class="close"]').click()

Doy click para ver la información de las tarifas

In [ ]:
vuelo.click()

In [ ]:
tarifas = vuelo.find_elements_by_xpath('.//div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]')

Extraigo la información de todas las tarifas

In [ ]:
precios = []
for tarifa in tarifas:
    nombre = tarifa.find_element_by_xpath('.//label').get_attribute('for')
    moneda = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="currency-symbol"]').text
    valor = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="value"]').text 
    dict_tarifa={nombre:{'moneda':moneda, 'valor':valor}}
    precios.append(dict_tarifa)
    print(dict_tarifa)

In [28]:
vuelo.click()

## Juntamos todo

### Función precios 

In [29]:
def obtener_precios(vuelo):
    tarifas = vuelo.find_elements_by_xpath(
        './/div[@class="fares-table-container"]//tfoot//td[contains(@class, "fare-")]')
    precios = []
    for tarifa in tarifas:
        nombre = tarifa.find_element_by_xpath('.//label').get_attribute('for')
        moneda = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="currency-symbol"]').text
        valor = tarifa.find_element_by_xpath('.//span[@class="price"]/span[@class="value"]').text 
        dict_tarifa={nombre:{'moneda':moneda, 'valor':valor}}
        precios.append(dict_tarifa)
    return precios

In [30]:
obtener_precios(vuelo) #Probamos

[{'LIGHT': {'moneda': 'ARS', 'valor': '50.403,38'}},
 {'PLUS': {'moneda': 'ARS', 'valor': '58.014,38'}},
 {'TOP': {'moneda': 'ARS', 'valor': '66.385,62'}}]

### Función escalas

In [25]:
def obtener_datos_escalas(vuelo):
    segmentos = vuelo.find_elements_by_xpath('//div[@class="sc-hZSUBg gfeULV"]/div[@class="sc-cLQEGU hyoued"]')
    info_escalas = []
    for segmento in segmentos:
        # Origen
        origen = segmento.find_element_by_xpath(
            './/div[@class="sc-bwCtUz iybVbT"]/abbr[@class="sc-hrWEMg hlCkST"]').text
        # Duración del vuelo
        duracion_vuelo = segmento.find_element_by_xpath('.//div[@class="sc-bwCtUz iybVbT"]/time').text

        # Armo un diccionario para almacenar los datos
        data_dict={'origen': origen, 
                    'duracion_vuelo': duracion_vuelo}
        info_escalas.append(data_dict)
        
    return info_escalas

In [26]:
obtener_datos_escalas(vuelo)

[{'origen': 'EZE', 'duracion_vuelo': '18:00'},
 {'origen': 'GRU', 'duracion_vuelo': '22:55'}]

### Función tiempos

In [14]:
def obtener_tiempos(vuelo):
    # Hora de salida
    salida = vuelo.find_element_by_xpath('.//div[@class="departure"]/time').get_attribute('datetime')
    # Hora de llegada
    llegada = vuelo.find_element_by_xpath('.//div[@class="arrival"]/time').get_attribute('datetime')
    # Duracion
    duracion = vuelo.find_element_by_xpath('.//span[@class="duration"]/time').get_attribute('datetime')
    return {'hora_salida': salida, 'hora_llegada': llegada, 'duracion': duracion}

In [15]:
obtener_tiempos(vuelo)

{'hora_salida': '18:00', 'hora_llegada': '14:10', 'duracion': 'PT15H10M'}

# Toda la información sobre un vuelo

In [33]:
def obtener_info(driver):
    vuelos = driver.find_elements_by_xpath('//li[@class="flight"]')
    print(f'Se encontraron {len(vuelos)} vuelos.')
    print('Iniciando scraping...')
    info = []
    for vuelo in tqdm(vuelos):
        # Obtenemos los tiempos generales del vuelo
        tiempos = obtener_tiempos(vuelo)
        # Clickeamos el botón de escalas para ver los detalles
        boton_escalas = vuelo.find_element_by_xpath('.//div[@class ="flight-summary-stops-description"]/button')
        boton_escalas.click()
        escalas = obtener_datos_escalas(vuelo)
        # Cerramos el pop-up con los detalles
        driver.find_element_by_xpath('//div[@class="modal-content sc-iwsKbI eHVGAN"]//button[@class="close"]').click()
        # Clickeamos el vuelo para ver los precios
        vuelo.click()
        precios = obtener_precios(vuelo)
        info.append({'precios':precios, 'tiempos':tiempos , 'escalas': escalas})
    return info

In [34]:
obtener_info(driver)

  0%|          | 0/43 [00:00<?, ?it/s]

Se encontraron 43 vuelos.
Iniciando scraping...


100%|██████████| 43/43 [01:52<00:00,  2.63s/it]


[{'precios': [{'LIGHT': {'moneda': 'ARS', 'valor': '50.403,38'}},
   {'PLUS': {'moneda': 'ARS', 'valor': '58.014,38'}},
   {'TOP': {'moneda': 'ARS', 'valor': '66.385,62'}}],
  'tiempos': {'hora_salida': '18:00',
   'hora_llegada': '14:10',
   'duracion': 'PT15H10M'},
  'escalas': [{'origen': 'EZE', 'duracion_vuelo': '18:00'},
   {'origen': 'GRU', 'duracion_vuelo': '22:55'}]},
 {'precios': [{'LIGHT': {'moneda': 'ARS', 'valor': '50.403,38'}},
   {'PLUS': {'moneda': 'ARS', 'valor': '58.014,38'}},
   {'TOP': {'moneda': 'ARS', 'valor': '66.385,62'}}],
  'tiempos': {'hora_salida': '15:35',
   'hora_llegada': '14:10',
   'duracion': 'PT17H35M'},
  'escalas': [{'origen': 'EZE', 'duracion_vuelo': '15:35'},
   {'origen': 'GRU', 'duracion_vuelo': '22:55'}]},
 {'precios': [{'LIGHT': {'moneda': 'ARS', 'valor': '50.403,38'}},
   {'PLUS': {'moneda': 'ARS', 'valor': '58.014,38'}},
   {'TOP': {'moneda': 'ARS', 'valor': '66.385,62'}}],
  'tiempos': {'hora_salida': '12:40',
   'hora_llegada': '12:50',
  

In [37]:
#Para cerrar navegador
driver.close()

# Introducir demoras

In [35]:
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.support import expected_conditions as EC #Espera a que se cumpla determinada condición
from selenium.webdriver.common.by import By 
from selenium.common.exceptions import TimeoutException

In [38]:
url='https://www.latam.com/es_ar/apps/personas/booking?fecha1_dia=17&fecha1_anomes=2021-04&fecha2_dia=24&fecha2_anomes=2021-09&otras_ciudades=&num_segmentos_interfaz=2&tipo_paso1=caja&rand_check=486.3230058787316&from_city2=MAD&to_city2=BUE&auAvailability=1&ida_vuelta=ida_vuelta&vuelos_origen=Buenos%20Aires,%20Argentina%20(BUE)&from_city1=BUE&vuelos_destino=Madrid,%20Espa%F1a%20(MAD)&to_city1=MAD&flex=1&vuelos_fecha_salida=27/DIC/2020&vuelos_fecha_salida_ddmmaaaa=27/12/2020&vuelos_fecha_regreso=21/ENE/2021&vuelos_fecha_regreso_ddmmaaaa=21/01/2021&cabina=Y&nadults=1&nchildren=0&ninfants=0&application=#/'
driver = webdriver.Safari()
driver.get(url)

#Introducir una demora
delay = 10
try:
    #Introducir una demora inteligente
    vuelo = WebDriverWait(driver, delay).until(EC.presence_of_element_located((By.XPATH, '//li[@class="flight"]')))
    print('La página terminó de cargar')
    info_vuelos = obtener_info(driver)
except TimeoutException:
    print('La página tardó demasiado en cargar')
    info_vuelos =[]
driver.close()

  0%|          | 0/43 [00:00<?, ?it/s]

La página terminó de cargar
Se encontraron 43 vuelos.
Iniciando scraping...


100%|██████████| 43/43 [01:44<00:00,  2.43s/it]


In [39]:
info_vuelos

[{'precios': [{'LIGHT': {'moneda': 'ARS', 'valor': '50.403,38'}},
   {'PLUS': {'moneda': 'ARS', 'valor': '58.014,38'}},
   {'TOP': {'moneda': 'ARS', 'valor': '66.385,62'}}],
  'tiempos': {'hora_salida': '18:00',
   'hora_llegada': '14:10',
   'duracion': 'PT15H10M'},
  'escalas': [{'origen': 'EZE', 'duracion_vuelo': '18:00'},
   {'origen': 'GRU', 'duracion_vuelo': '22:55'}]},
 {'precios': [{'LIGHT': {'moneda': 'ARS', 'valor': '50.403,38'}},
   {'PLUS': {'moneda': 'ARS', 'valor': '58.014,38'}},
   {'TOP': {'moneda': 'ARS', 'valor': '66.385,62'}}],
  'tiempos': {'hora_salida': '15:35',
   'hora_llegada': '14:10',
   'duracion': 'PT17H35M'},
  'escalas': [{'origen': 'EZE', 'duracion_vuelo': '15:35'},
   {'origen': 'GRU', 'duracion_vuelo': '22:55'}]},
 {'precios': [{'LIGHT': {'moneda': 'ARS', 'valor': '50.403,38'}},
   {'PLUS': {'moneda': 'ARS', 'valor': '58.014,38'}},
   {'TOP': {'moneda': 'ARS', 'valor': '66.385,62'}}],
  'tiempos': {'hora_salida': '12:40',
   'hora_llegada': '12:50',
  